In [3]:
import numpy as np
import pandas as pd
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import statsmodels.api as sm

## investigating volatility linkages between oil, gas, and regionalised coal futures markets ##

## load data from Github into new frame and remove rows with absent data ##

url = 'https://raw.githubusercontent.com/joe-ascroft/phd/master/df-combined.csv'
df = pd.read_csv(url)
df["DATE"] = pd.to_datetime(df["DATE"],dayfirst=True)
for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col],errors='coerce')

df = df.dropna()
print(df)
df.to_csv('df_cleared.csv')

           DATE    NCF    RCF    WTI    HH  Brent  NBP_UK  GBP-USD       NBP
1    2009-01-02  74.35  81.00  46.17  5.41  46.91   54.74   1.4453  7.911572
4    2009-01-05  77.65  84.40  48.61  5.83  49.62   56.66   1.4708  8.333553
5    2009-01-06  81.25  88.50  48.56  6.10  50.53   59.88   1.4919  8.933497
6    2009-01-07  78.65  86.25  42.75  5.89  45.86   58.03   1.5111  8.768913
7    2009-01-08  79.15  84.00  41.68  5.96  44.67   55.74   1.5202  8.473595
...         ...    ...    ...    ...   ...    ...     ...      ...       ...
4254 2020-08-25  49.75  48.90  43.17  2.54  45.86   24.61   1.3148  3.235723
4255 2020-08-26  49.70  48.95  43.21  2.52  45.64   25.10   1.3193  3.311443
4256 2020-08-27  49.80  48.90  42.88  2.52  45.09   26.68   1.3206  3.523361
4257 2020-08-28  50.60  52.20  42.96  2.46  45.81   28.95   1.3344  3.863088
4260 2020-08-31  51.95  52.90  42.61  2.30  45.28   28.95   1.3379  3.873220

[2856 rows x 9 columns]


In [ ]:
plt.plot(df["DATE"],df["RCF-NBP"])

In [4]:
## markov regime switching for Henry Hub and Newcastle Gas Futures##

mod = sm.tsa.MarkovRegression(df['WTI'], k_regimes=5, order=4)
res_mod = mod.fit()
res_mod.summary()

/Users/joe/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/joe/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                        Markov Switching Model Results                        
==============================================================================
Dep. Variable:                    WTI   No. Observations:                 2856
Model:               MarkovRegression   Log Likelihood               -8977.449
Date:                Wed, 24 Mar 2021   AIC                          18006.898
Time:                        15:05:01   BIC                          18161.785
Sample:                             0   HQIC                         18062.750
                               - 2856                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         31.7045      1.248     25.405      0.000      29.259      34.150
                             Regime 1 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         48.3710      0.422    114.550      0.000      47.543      49.199
                             Regime 2 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         63.0197      0.341    185.054      0.000      62.352      63.687
                             Regime 3 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         81.6480      0.284    287.309      0.000      81.091      82.205
                             Regime 4 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         99.1185      0.215    461.783      0.000      98.698      99.539
                           Non-switching parameters                           
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2        28.1463      0.761     36.962      0.000      26.654      29.639
                         Regime transition parameters                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p[0->0]        0.9825      0.150      6.557      0.000       0.689       1.276
p[1->0]        0.0028      0.002      1.637      0.102      -0.001       0.006
p[2->0]        0.0007      0.031      0.023      0.982      -0.060       0.062
p[3->0]        0.0011      0.089      0.013      0.990      -0.174       0.176
p[4->0]     2.116e-07        nan        nan        nan         nan         nan
p[0->1]        0.0169      0.010      1.636      0.102      -0.003       0.037
p[1->1]        0.9883      0.033     29.866      0.000       0.923       1.053
p[

In [5]:
res.smoothed_marginal_probabilities[3].plot(
    title='Probability of being in the high regime', figsize=(12,2))

NameError: name 'res' is not defined

In [ ]:
## OLS on volatility output ##

X = df['NBP']
y = df['WTI']
X = sm.add_constant(X)
mod1 = sm.OLS(y,X)
results = mod1.fit()
print(results.summary())

In [ ]:
sm.qqplot(results.resid,line="45",fit="TRUE")